In [1]:
import numpy as np
from fractions import gcd

In [2]:
def get_min_val(N, diag):
    """Finds the minimum value of a matrix that's not in a row or
    column of zeros.
    
    Args:
        N (list): The input matrix.
        diag (int): Specifiec index for submatrix.
    Returns:
        The minimum value in the matrix that isn't already in
        a zeroed row or column."""
    
    temp_N = abs(N)
    list_N = sorted(temp_N.flatten())
    for v in list_N:
        if v == 0:
            continue
        row, col = np.argwhere(temp_N==v)[0]
        if ((row >= diag and col >= diag)):
            return N[row,col], row, col
        else:
            temp_N[row,col] = -1

In [3]:
def get_min_loc(M, S, diag):
    """Finds the location of the maximum value of a matrix that's not in a row or
    column of zeros.
    
    Args:
        M (list): The modulus matrix.
        S (list): The input matrix.
        diag (int): Specifiec index for submatrix.
    Returns:
        The location of the maximum value in the matrix that isn't already in
        a zeroed row or column."""
    
    temp_S = abs(S)
    list_S = sorted(temp_S.flatten())
    for v in list_S:
        if v == 0:
            continue
        row, col = np.argwhere(temp_S==v)[0]
        if (row >= diag and col >= diag and M[row,col]!=0):
            return row
        else:
            temp_S[row,col] = -1

In [37]:
def Rods_way(N):
    """Finds the SmithNormalForm by reducing starting with the 
    smallest entry in the matrix.
    
    Args:
        N (list): An integer 3x3 matrix.
        
    Returns:
        L,S,R: the left transform, the SNF and the right 
        transform.
    """
    from copy import deepcopy
    S = np.array(N)
    L = np.identity(3)
    R = np.identity(3)
    
    is_snf = False
    cur_diag = 0
    count = 0
    new_pivot = True
    # First get the top right corner correct. Steps 1 and 2
    while (not is_snf) and count<100:
        count += 1
        # IF we need a new pivot find it step 1
        if new_pivot:
            min_val, row, col = get_min_val(S, cur_diag)
            
        print("count", count)
        print("S",S)
        print("L",L)
        print("R",R)
        print("curr diag", cur_diag)
        print("min_val", min_val, "row", row, "col", col)
        # step 2
        #reduce the column
        for j in range(3):
            if j == col:
                continue
            multiple = int(np.round(S[row,j]/min_val))
            if multiple==0:
                continue
            S[:,j] = S[:,j]-multiple*S[:,col]
            R[:,j] = R[:,j]-multiple*R[:,col]
            
        #then reduce the row
        for j in range(3):
            print("reducing row: ",j)            
            if j == row:
                continue
            multiple = int(np.round(S[j,col]/min_val))
            if multiple==0:
                continue
            print("multiple",multiple)
            S[j,:] = S[j,:]-multiple*S[row,:]
            L[j,:] = L[j,:]-multiple*L[row,:]
        
        print("S2",S)
        print("L",L)
        print("R",R)
        # Determine which case of 2a-2c we have.
        new_pivot=True
        if ((list(S[row,:]).count(0)==2) and 
            list(S[:,col]).count(0)==2): # If this condition is false the we have case 
                                         # 2a and need a new pivot
            # This is either b or c.
            if (np.allclose(S[cur_diag:,cur_diag:]%min_val,0)):
                #This is 2c move the entry to the top left 
                if cur_diag < col:
                    #Swap rows and columns
                    tmp_col = deepcopy(S[:,cur_diag])
                    S[:,cur_diag] = deepcopy(S[:,col])
                    S[:,col] = tmp_col
                    tmp_col = deepcopy(R[:,cur_diag])
                    R[:,cur_diag] = deepcopy(R[:,col]) 
                    R[:,col] = tmp_col
                if cur_diag < row:
                    tmp_row = deepcopy(S[cur_diag,:])
                    S[cur_diag,:] = deepcopy(S[row,:])
                    S[row,:] = tmp_row
                    tmp_row = deepcopy(L[cur_diag,:])
                    L[cur_diag,:] = deepcopy(L[row,:])
                    L[row,:] = tmp_row
                cur_diag += 1
            else:
                # This is 2b, find the smallest entry that the pivot
                # doesn't divide and add it's row to the pivot row
                mods = S%min_val
                new_pivot = False
                min_row = get_min_loc(mods, S, cur_diag)
                S[row,:] = S[row,:] + S[min_row,:]
                L[row,:] = L[row,:] + L[min_row,:]  
        print("S3",S)
        print("L",L)
        print("R",R)
        
        if (np.allclose([S[0][1],S[0][2],S[1][0],S[1][2],
                         S[2][0],S[2][1]], 0) and 
            S[1][1]%S[0][0]==0 and S[2][2]%S[1][1]==0):
            is_snf = True
            
    for j in range(3):
        if S[j,j] < 0:
            S[j,:] = -S[j,:]
            L[j,:] = -L[j,:]
            
    if count == 100:
        print("Failed to find SNF in 100 iterations.")
    if not np.allclose(np.matmul(np.matmul(L,N),R),S):
        print("Transformation failed in SNF.")
            
    return L, S, R

In [38]:
np.set_printoptions(suppress=True)
#N = np.array([[63,0,0],[0,1,0],[0,424,1175]])
N = np.array([[2,0,0],[0,9,0],[120,7,139]])
L,S,R = Rods_way(N)
print(L)
print(S)
print(R)

('count', 1)
('S', array([[  2,   0,   0],
       [  0,   9,   0],
       [120,   7, 139]]))
('L', array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]]))
('R', array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]]))
('curr diag', 0)
('min_val', 2, 'row', 0, 'col', 0)
('reducing row: ', 0)
('reducing row: ', 1)
('reducing row: ', 2)
('multiple', 60)
('S2', array([[  2,   0,   0],
       [  0,   9,   0],
       [  0,   7, 139]]))
('L', array([[  1.,   0.,   0.],
       [  0.,   1.,   0.],
       [-60.,   0.,   1.]]))
('R', array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]]))
('S3', array([[  2,   7, 139],
       [  0,   9,   0],
       [  0,   7, 139]]))
('L', array([[-59.,   0.,   1.],
       [  0.,   1.,   0.],
       [-60.,   0.,   1.]]))
('R', array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]]))
('count', 2)
('S', array([[  2,   7, 139],
       [  0,   9,   0],
       [  0,   7, 139]]))
('L', array([[-59.,

In [32]:
np.matmul(np.matmul(L,N),R)


array([[    1.,     0.,     0.],
       [    0.,     1.,     0.],
       [    0.,     0.,  2502.]])

In [194]:
np.round(np.linalg.det(N))

2502.0

In [64]:
list(reversed(range(3)))

[2, 1, 0]

In [134]:
False and False or True

True

In [192]:
np.round(np.linalg.det([[-1,0,0],[0,2,0],[0,-1244,2491]]))

-4982.0

In [29]:
int(abs(-132)//9.)*np.sign(-132)

-14